# REDDIT dataset benchmark



## References


## Description
This notebook will experiment with:
- train & compare differnt GNN models for graph classificaiton in common benchmarks (Enzymes)
- compare results to publication results



## Model training experiments

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing
#from torch_geometric.nn.conv.gated_graph_conv import GatedGraphConv
from torch_geometric.nn.glob.glob import global_mean_pool, global_add_pool
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer
from torch_geometric.datasets import TUDataset, QM9, QM7b, PPI, Planetoid, KarateClub
import copy

from TFM_graph_classification import *
from TFM_graph_classification_models import *

global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
thename='REDDIT-BINARY'
dataset = TUDataset(root='/tmp/'+thename, name=thename)
dataset = dataset.shuffle()
num_classes=dataset.num_classes

k=3

train_dataset, test_dataset = prepare_dataset(dataset, nfolds=k, prop=0.8, 
                                              dataset_type="balanced", print_debug=True)

 n: 1113  k folds= 3
Datasets balancing: 
{0: 663, 1: 450}
{0: 530, 1: 360}
{0: 133, 1: 90}



In [3]:
resultsdict = {'best_models_list':[], 'models':{} , 'best_models':{}, 'autoincrement': 0 }

In [4]:
model_list21 = hpsearch(None, {
    'd1': [25,50],
    'd2': [20,50],
    'epochs': [300,600],
    'model': [GGNN1],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'num_layers':[2,4,8],
    'aggr_type': ['mean','add'],
    'batch_size': [32]
})
modelsdict1 = modelSelection(model_list21,k, train_dataset, balanced=False)
reportModelSelectionResult(modelsdict1,resultsdict)

 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 300  val loss= 0.01902580385406812  val accuracy= 0.9166666666666666  val microF1= 0.7443693693693693  val macroF1= 0.7325477812955223
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 300  val loss= 0.019560431440671284  val accuracy= 0.8333333333333334  val microF1= 0.731981981981982  val macroF1= 0.7198286784367262
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 300  val loss= 0.01875387318432331  val accuracy= 0.7916666666666666  val microF1= 0.731981981981982  val macroF1= 0.7221045425285587
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 300  val loss= 0.019788247843583424  val accuracy= 0.7083333333333334  val microF1= 0.7038288288288288  val macroF1= 0.6881855893880328
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 300  va

 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 300  val loss= 0.020882181202371914  val accuracy= 0.625  val microF1= 0.6745495495495496  val macroF1= 0.5955343069575415
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 300  val loss= 0.02083258330821991  val accuracy= 0.75  val microF1= 0.6745495495495496  val macroF1= 0.6559880977633653
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 300  val loss= 0.022977035492658615  val accuracy= 0.4166666666666667  val microF1= 0.5945945945945946  val macroF1= 0.37288135593220345
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 8, 'aggr_type': 'add'}  epochs: 300  val loss= 0.022644175216555595  val accuracy= 0.75  val microF1= 0.5945945945945946  val macroF1= 0.37288135593220345
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 600  val loss= 0.01878446340560913  val acc

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.625,1,0.75,0.75
batch_size,32,32,32,32
cv_val_accuracy,0.708333,0.916667,0.625,0.625
cv_val_loss,0.0186705,0.0190258,0.0192764,0.0192764
cv_val_macroF1,0.724808,0.732548,0.735332,0.735332
cv_val_microF1,0.739865,0.744369,0.746622,0.746622
epochs,300,300,300,300
filename,0_GGNN1__d1-25_d2-50_num_layers-4_aggr_type-ad...,1_GGNN1__d1-25_d2-20_num_layers-2_aggr_type-me...,3_GGNN1__d1-50_d2-50_num_layers-2_aggr_type-me...,3_GGNN1__d1-50_d2-50_num_layers-2_aggr_type-me...
final_model,"GGNN1(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN1(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN1(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN1(\n (ggnn): GatedGraphConv(50, num_layer..."
kwargs,"{'d1': 25, 'd2': 50, 'num_layers': 4, 'aggr_ty...","{'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_ty...","{'d1': 50, 'd2': 50, 'num_layers': 2, 'aggr_ty...","{'d1': 50, 'd2': 50, 'num_layers': 2, 'aggr_ty..."


In [5]:
model_list22 = hpsearch(None, {
    'model': [GGNN2],
    'd1': [25,50],
    'd2': [20,50],
    'aggr_type': ['mean','add'],
    'epochs': [300,600],
    'num_layers':[2,4,8],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [32]
})
modelsdict2 = modelSelection(model_list22,k, train_dataset)
reportModelSelectionResult(modelsdict2,resultsdict)

 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 300  val loss= 0.021091913804411888  val accuracy= 0.6583333333333333  val microF1= 0.7016190719020908  val macroF1= 0.7064619133903692
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.02277625600496928  val accuracy= 0.6333333333333333  val microF1= 0.5765553289138196  val macroF1= 0.6047999846900342
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.022631095101435978  val accuracy= 0.5  val microF1= 0.6326279109297978  val macroF1= 0.6490321015684088
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.020946493993202846  val accuracy= 0.7083333333333334  val microF1= 0.7076746557878634  val macroF1= 0.6957053012247894
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 600  val loss= 0.

 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.0215054868410031  val accuracy= 0.5916666666666667  val microF1= 0.6360487846336903  val macroF1= 0.6268946653162524
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.02415141649544239  val accuracy= 0.5416666666666666  val microF1= 0.4688721740608533  val macroF1= 0.39438343855226504
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.02046545222401619  val accuracy= 0.725  val microF1= 0.7097144314125446  val macroF1= 0.7169855318543092
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 600  val loss= 0.02099430312712987  val accuracy= 0.7666666666666666  val microF1= 0.6806476287608364  val macroF1= 0.6545366031026453
 trained model:  GGNN2 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 600  val loss= 0.0

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.8,0.75,0.85,0.8
batch_size,32,32,32,32
cv_val_accuracy,0.808333,0.833333,0.783333,0.725
cv_val_loss,0.0195343,0.0208381,0.0204349,0.0204655
cv_val_macroF1,0.667071,0.684262,0.70792,0.716986
cv_val_microF1,0.673253,0.686512,0.712859,0.709714
epochs,300,600,600,600
filename,4_GGNN2__d1-25_d2-20_aggr_type-add_num_layers-...,5_GGNN2__d1-25_d2-20_aggr_type-add_num_layers-...,6_GGNN2__d1-25_d2-20_aggr_type-add_num_layers-...,7_GGNN2__d1-50_d2-50_aggr_type-mean_num_layers...
final_model,"GGNN2(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN2(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN2(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN2(\n (ggnn): GatedGraphConv(50, num_layer..."
kwargs,"{'d1': 25, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 25, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 25, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num..."


In [6]:
model_list23 = hpsearch(None, {
    'model': [GGNN3],
    'd1': [25,50],
    'd2': [20,50],
    'aggr_type': ['mean','add'],
    'epochs': [300,600],
    'num_layers':[2,4,8],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [32]
})
modelsdict3 = modelSelection(model_list23,k, train_dataset)
reportModelSelectionResult(modelsdict3,resultsdict)

 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 300  val loss= 0.019418294231096905  val accuracy= 0.6833333333333332  val microF1= 0.7042537820839708  val macroF1= 0.6927495639253275
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.022124049564202625  val accuracy= 0.7083333333333334  val microF1= 0.6828573856875743  val macroF1= 0.6815648696935656
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.026960138852397602  val accuracy= 0.6333333333333333  val microF1= 0.5849481557028727  val macroF1= 0.5216970419803441
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.02170936328669389  val accuracy= 0.6583333333333333  val microF1= 0.7046999830018699  val macroF1= 0.7043022515478873
 trained model:  GGNN3 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 60

 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.02200466642777125  val accuracy= 0.7416666666666667  val microF1= 0.6549592044875063  val macroF1= 0.6814069106482646
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.027152183776100475  val accuracy= 0.6416666666666667  val microF1= 0.5959119496855346  val macroF1= 0.46561841568909373
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.020871015265583992  val accuracy= 0.6916666666666668  val microF1= 0.6779279279279279  val macroF1= 0.6784000024426935
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 600  val loss= 0.020990317066510517  val accuracy= 0.7000000000000001  val microF1= 0.6758669046404896  val macroF1= 0.6804750426873564
 trained model:  GGNN3 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 6

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.7,0.75,0.8,0.8
batch_size,32,32,32,32
cv_val_accuracy,0.691667,0.833333,0.766667,0.766667
cv_val_loss,0.0191789,0.0200212,0.0192469,0.0192469
cv_val_macroF1,0.710308,0.709795,0.730267,0.730267
cv_val_microF1,0.723632,0.714856,0.740056,0.740056
epochs,600,600,300,300
filename,8_GGNN3__d1-25_d2-50_aggr_type-add_num_layers-...,9_GGNN3__d1-50_d2-20_aggr_type-mean_num_layers...,11_GGNN3__d1-50_d2-20_aggr_type-add_num_layers...,11_GGNN3__d1-50_d2-20_aggr_type-add_num_layers...
final_model,"GGNN3(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN3(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN3(\n (ggnn): GatedGraphConv(50, num_layer...","GGNN3(\n (ggnn): GatedGraphConv(50, num_layer..."
kwargs,"{'d1': 25, 'd2': 50, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 20, 'aggr_type': 'mean', 'num...","{'d1': 50, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 20, 'aggr_type': 'add', 'num_..."


In [7]:
model_list24 = hpsearch(None, {
    'model': [GGNN4],
    'd1': [25,50],
    'd2': [20,50],
    'aggr_type': ['mean','add'],
    'epochs': [300,600],
    'num_layers':[2,4,8],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [32]
})
modelsdict4 = modelSelection(model_list24,k, train_dataset)
reportModelSelectionResult(modelsdict4,resultsdict)

 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 300  val loss= 0.019066612236201763  val accuracy= 0.7416666666666667  val microF1= 0.7404810470848208  val macroF1= 0.7317461386948022
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.020433853690822918  val accuracy= 0.8250000000000001  val microF1= 0.6999617542070373  val macroF1= 0.6996072451214547
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.028429288417100906  val accuracy= 0.6  val microF1= 0.5660377358490566  val macroF1= 0.3592592592592592
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.019152355069915455  val accuracy= 0.8083333333333332  val microF1= 0.733745537990821  val macroF1= 0.7265627592315148
 trained model:  GGNN4 {'d1': 25, 'd2': 20, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 600  val loss= 0.

 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 300  val loss= 0.04830314281086127  val accuracy= 0.325  val microF1= 0.49026857045724964  val macroF1= 0.41866612777342355
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 300  val loss= 0.0272798091173172  val accuracy= 0.7000000000000001  val microF1= 0.5660377358490566  val macroF1= 0.3592592592592592
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 2}  epochs: 600  val loss= 0.020609548936287563  val accuracy= 0.7083333333333334  val microF1= 0.7087582865884753  val macroF1= 0.7051961971857552
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 4}  epochs: 600  val loss= 0.024594102675716083  val accuracy= 0.4583333333333333  val microF1= 0.5535441101478837  val macroF1= 0.6307834220171752
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'aggr_type': 'mean', 'num_layers': 8}  epochs: 600  val loss= 

,best_model_loss,best_model_acc,best_model_microF1,best_model_macroF1
accuracy,0.8,0.8,0.8,0.75
batch_size,32,32,32,32
cv_val_accuracy,0.85,0.85,0.85,0.75
cv_val_loss,0.018647,0.018647,0.018647,0.0190701
cv_val_macroF1,0.734894,0.734894,0.734894,0.738538
cv_val_microF1,0.745687,0.745687,0.745687,0.740757
epochs,300,300,300,300
filename,14_GGNN4__d1-25_d2-20_aggr_type-add_num_layers...,14_GGNN4__d1-25_d2-20_aggr_type-add_num_layers...,14_GGNN4__d1-25_d2-20_aggr_type-add_num_layers...,15_GGNN4__d1-50_d2-50_aggr_type-add_num_layers...
final_model,"GGNN4(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN4(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN4(\n (ggnn): GatedGraphConv(25, num_layer...","GGNN4(\n (ggnn): GatedGraphConv(50, num_layer..."
kwargs,"{'d1': 25, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 25, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 25, 'd2': 20, 'aggr_type': 'add', 'num_...","{'d1': 50, 'd2': 50, 'aggr_type': 'add', 'num_..."


In [ ]:
model_list25 = hpsearch(None, {
    'model': [META3,],
    'd1': [3],
    'd2': [10,20,50],
    'd3':[15],
    'd4':[15],
    'd5':[10,20],
    'epochs': [300,600],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [64],
    'num_classes': [num_classes]
})
modelsdict5 = modelSelection(model_list25,k, train_dataset)
reportModelSelectionResult(modelsdict5,resultsdict)

 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10, 'num_classes': 2}  epochs: 300  val loss= 0.009389634865025679  val accuracy= 0.7916666666666666  val microF1= 0.7422870984191738  val macroF1= 0.738559190891512


In [ ]:
model_list26 = hpsearch(None, {
    'model': [META2,],
    'd1': [3],
    'd2': [10,20,50],
    'd3':[15],
    'd4':[15],
    'd5':[10,20],
    'epochs': [300,600],
    'learning_rate': [0.01],
    'weight_decay':[5e-4],
    'batch_size': [64],
    'num_classes': [num_classes]
})
modelsdict6 = modelSelection(model_list26,k, train_dataset, debug_training=False)
reportModelSelectionResult(modelsdict6, resultsdict)

## Testing

In [4]:
test_multiple_models(resultsdict, test_dataset)

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #2 'mat2'

## Final results report
We have obtained a **?%** to **?%** accuracy on dataset with published accuracy of a **72.33%** to **84.08%**.


In [11]:
report_all_test(resultsdict)

2019-06-26_10-07-30


,accuracy,macroF1,microF1,name
0,0.6816,0.6636,0.6816,16_META3
1,0.7040,0.6841,0.7040,20_META2
2,0.6682,0.6607,0.6682,3_GGNN1
3,0.7130,0.7030,0.7130,8_GGNN3
4,0.6906,0.6731,0.6906,0_GGNN1
5,0.6951,0.6761,0.6951,7_GGNN2
6,0.6996,0.6786,0.6996,15_GGNN4
7,0.6413,0.6361,0.6413,11_GGNN3
8,0.7040,0.6887,0.7040,19_META3
9,0.6861,0.6640,0.6861,6_GGNN2


## Reload models from disk

In [3]:
filenames =[
    './models/enzymes2/0_GGNN1__d1-25_d2-50_num_layers-4_aggr_type-add_300_0.01_0.0005_32_date2019-06-26_01-15-24',
]

#resultsdict_old = copy.deepcopy(resultsdict)
resultsdict = {'best_models_list':[], 'models':[] , 'best_models':{}, 'testing': {}, 'autoincrement': 0 }
for path in filenames:
    modeldict = loadModelFromFile(path)
    resultsdict['models'].append(modeldict)
    resultsdict['best_models_list'].append(modeldict)


In [4]:
test_multiple_models(resultsdict, test_dataset)
report_all_test(resultsdict)

0_GGNN1
./models/proteins2/0_GGNN1__d1-25_d2-50_num_layers-4_aggr_type-add_300_0.01_0.0005_32_date2019-06-26_01-15-24
{0, 1}
{0, 1}
num_classes 2
11_GGNN3
./models/proteins2/11_GGNN3__d1-50_d2-20_aggr_type-add_num_layers-2_300_0.01_0.0005_32_date2019-06-26_05-43-00
{0, 1}
{0, 1}
num_classes 2
14_GGNN4
./models/proteins2/14_GGNN4__d1-25_d2-20_aggr_type-add_num_layers-2_300_0.01_0.0005_32_date2019-06-26_07-55-44
{0, 1}
{0, 1}
num_classes 2
15_GGNN4
./models/proteins2/15_GGNN4__d1-50_d2-50_aggr_type-add_num_layers-2_300_0.01_0.0005_32_date2019-06-26_07-55-44
{0, 1}
{0, 1}
num_classes 2
16_META3
./models/proteins2/16_META3__d1-3_d2-50_d3-15_d4-15_d5-20_600_0.01_0.0005_64_date2019-06-26_09-46-38
{0}
{0, 1}
num_classes 2
17_META3
./models/proteins2/17_META3__d1-3_d2-20_d3-15_d4-15_d5-20_600_0.01_0.0005_64_date2019-06-26_09-46-38
{0, 1}
{0, 1}
num_classes 2
19_META3
./models/proteins2/19_META3__d1-3_d2-20_d3-15_d4-15_d5-10_300_0.01_0.0005_64_date2019-06-26_09-46-38
{0, 1}
{0, 1}
num_classes 2

,accuracy,macroF1,microF1,name
8,0.0852,0.0533,0.0852,20_META2
4,0.5964,0.3736,0.5964,16_META3
12,0.6502,0.6468,0.6502,5_GGNN2
14,0.6771,0.6846,0.6771,7_GGNN2
3,0.6771,0.6810,0.6771,15_GGNN4
11,0.6816,0.6699,0.6816,4_GGNN2
9,0.6861,0.6764,0.6861,23_META2
16,0.6861,0.6717,0.6861,9_GGNN3
15,0.6996,0.6786,0.6996,8_GGNN3
2,0.7085,0.6915,0.7085,14_GGNN4
